In [155]:
import pandas as pd
import numpy as np
import math as mt

from rich.console import Console
from rich.table import Table 

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances

In [156]:
#from dataframe.cores import *

# Declara a classe cores
#cores = cores()

In [157]:
from IPython.display import display

In [158]:
df = pd.read_csv('Resposta.csv', sep=',')

In [159]:
display(df)

,1,2,3,4,5,5.1,5.2,5.3,5.4,5.5,...,20,20.1,20.2,20.3,20.4,20.5,20.6,21,22,23
0,0.0,0.0,0.75,0.00,1,1,1,0,0,0,...,0,0,1,0,1,0,0,0.0,0.0,0.0
1,0.0,0.0,0.25,0.00,1,0,1,1,0,1,...,1,0,1,0,0,0,1,0.5,0.0,0.0
2,0.0,0.0,0.25,0.00,1,0,0,0,0,1,...,1,1,0,0,1,0,0,0.5,0.0,0.0
3,0.0,0.0,0.50,0.00,1,0,0,0,0,0,...,0,0,1,1,0,1,0,0.0,0.0,0.0
4,0.0,0.0,0.75,0.00,1,1,0,0,0,0,...,1,1,1,1,0,1,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0.5,0.0,0.25,0.16,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0.5,0.5,0.0
179,0.5,0.0,0.25,0.16,0,1,0,0,0,0,...,1,1,1,0,1,1,0,0.0,0.0,0.0
180,0.5,0.0,0.25,0.00,0,1,1,0,0,0,...,1,0,0,1,1,1,0,0.0,0.0,0.5
181,0.5,0.0,0.25,0.60,0,0,1,0,0,0,...,1,0,1,1,1,1,0,0.5,0.0,1.0


---

# Função de Otimização do KMeans

Define uma função ideal para o kmeans que executa clustering K-means em um determinado DataFrame df para um intervalo de números de cluster e visualiza os resultados. A função leva três argumentos: df (o DataFrame a ser agrupado), start (o número inicial de clusters a testar) e end (o número final de clusters a testar). Resultando em todos os possiveis valores de clusters e suas respectivas inércias.

In [160]:
# Esta função plota a soma dos erros quadráticos (SSE) e o escore de silhueta para um intervalo de clusters.
def optimal_kmeans(df, start=2, end=11):
    
    '''
    df: DataFrame que sera clusterizado.
    start: Número inicial de clusters que você deseja testar.
    end: Número final de clusters que você deseja testar.
    '''
    
    # Soma dos erros quadráticos
    sse = {}
    
    # Escore de silhueta
    silhouette = {}
    
    # Adiciona um dicionario de resultados
    results = {}
    
    for k in range(start, end):
        kmeans = KMeans(n_clusters=k, max_iter=100, random_state=42).fit(df)
        df["clusters"] = kmeans.labels_
        sse[k] = kmeans.inertia_ 
        silhouette[k] = silhouette_score(df, kmeans.labels_, metric='euclidean')
      

    # Plota um subplot com o plotly com o primeiro gráfico sendo o sse e o outro com o silouette
    fig = make_subplots(rows=1, cols=2, subplot_titles=("SSE", "Silhouette Score"))

    fig.add_trace(go.Scatter(x=list(sse.keys()), y=list(sse.values()), 
                             name='SSE'), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=list(silhouette.keys()), y=list(silhouette.values()), 
                             name='Silhouette Score'), row=1, col=2)
    
    fig.update_layout(title_text="Gráfico de análise - SSE e Silhouette Score")

    fig.update_layout(
        autosize=False,
        width=1800,
        height=800,
    )    

    fig.show()
    
    # Com o rich, cria uma tabela com os resultados
    table = Table(title="Resultados")
    table.add_column("Clusters", justify="center", style="cyan", no_wrap=True)
    table.add_column("SSE", justify="center", style="magenta", no_wrap=True)
    table.add_column("Silhouette Score", justify="center", style="green", no_wrap=True)
    
    for k in range(start, end):
        table.add_row(str(k), str(sse[k]), str(silhouette[k]))

    console = Console()
    console.print(table)

# Uso da função
optimal_kmeans(df)

                      Resultados                      
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Clusters ┃        SSE        ┃  Silhouette Score   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│    2     │ 9576.74104527417  │ 0.16920621358962182 │
│    3     │ 8341.46793351403  │ 0.16249737641924025 │
│    4     │ 7785.50558899986  │ 0.14072180249012728 │
│    5     │ 7189.363893201829 │ 0.14869392249612312 │
│    6     │ 7206.517955397879 │ 0.13578003816409026 │
│    7     │ 6652.85955183162  │ 0.16076565609253277 │
│    8     │ 6429.82610512333  │ 0.15930815675900908 │
│    9     │ 6200.957610981566 │ 0.15969064702485514 │
│    10    │ 6151.469383759534 │ 0.14734541980208135 │
└──────────┴───────────────────┴─────────────────────┘

# Função de Otimização do DBSCAN

Define uma função ideal para o dbscan que é usada para encontrar os parâmetros ideais para o algoritmo de cluster DBSCAN (Density-Based Spatial Clustering of Applications with Noise). A função leva um DataFrame df para ser agrupado, um intervalo de eps_values (distância máxima entre duas amostras para que elas sejam consideradas na mesma vizinhança) e min_samples_values (o número de amostras em uma vizinhança para que um ponto seja considerado como um ponto central) como entradas. Realizando uma busca em grade, mostrando todos os valores possiveis

In [161]:
# Esta função plota o escore de silhueta para uma variedade de valores eps e min_samples.
def optimal_dbscan(df, eps_values=np.linspace(0.1, 20, 50), min_samples_values=range(2, 15)):
    
    '''
    df: DataFrame que sera clusterizado.
    eps_values: Valores de eps que você deseja testar.
    min_samples_values: Valores de min_samples que você deseja testar.
    '''
    
    # Escore de silhueta
    silhouette = {}
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(df)
            n_clusters = len(set(dbscan.labels_))
            if 1 < n_clusters < df.shape[0]:  # Deve haver pelo menos 2 clusters e menos que n_samples para o escore de silhueta
                silhouette[(eps, min_samples)] = silhouette_score(df, dbscan.labels_)
    
    # Plota um gráfico com o plotly
    fig = go.Figure(data=go.Scatter(x=[str(i) for i in silhouette.keys()], y=list(silhouette.values())))
    
    fig.update_layout(title_text="Gráfico de análise - DBSCAN")

    fig.update_layout(
        autosize=False,
        width=1400,
        height=1000,
    )
    
    fig.show()
    
    # Com o rich, cria uma tabela com os resultados
    table = Table(title="Resultados")
    table.add_column("Eps", justify="center", style="cyan", no_wrap=True)
    table.add_column("Min Samples", justify="center", style="magenta", no_wrap=True)
    table.add_column("Silhouette Score", justify="center", style="green", no_wrap=True)
    
    for key, value in silhouette.items():
        table.add_row(str(key[0]), str(key[1]), str(value))
    
    console = Console()
    console.print(table)
    
optimal_dbscan(df)

                         Resultados                          
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Eps         ┃ Min Samples ┃    Silhouette Score    ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 3.3489795918367347 │      2      │  -0.10480444112901122  │
│ 3.7551020408163267 │      2      │  -0.10480444112901122  │
│ 4.161224489795918  │      2      │  -0.22596111074857672  │
│ 4.5673469387755095 │      2      │  -0.22909419142284043  │
│ 4.5673469387755095 │      3      │  -0.11208135736905296  │
│ 4.5673469387755095 │      4      │  -0.04155962462632482  │
│ 4.9734693877551015 │      2      │  -0.17716011858548572  │
│ 4.9734693877551015 │      3      │  -0.17245818638727373  │
│ 4.9734693877551015 │      4      │ -0.016375654662300506  │
│ 4.9734693877551015 │      5      │  -0.01992745662180044  │
│ 5.3795918367346935 │      2      │  -0.07778250141686645  │
│ 5.3795918367346935 │      3      │  -0.07352363627451836  │
│ 5.3795918367346935 │      4      │  -0.11392868032410289  │
│ 5.3795918367346935 │      5      │  -0.0942986551512442   │
│ 5.3795918367346935 │      6      │  -0.07635804121647895  │
│ 5.3795918367346935 │      7      │ 1.1842172408340404e-05 │
│ 5.3795918367346935 │      8      │ 1.1842172408340404e-05 │
│ 5.785714285714285  │      2      │ -0.008169765445336931  │
│ 5.785714285714285  │      3      │  0.003197909763609542  │
│ 5.785714285714285  │      4      │ -0.015806510398767574  │
│ 5.785714285714285  │      5      │ -0.023890499426630362  │
│ 5.785714285714285  │      6      │ -0.008611292727944827  │
│ 5.785714285714285  │      7      │  -0.05769213905651315  │
│ 5.785714285714285  │      8      │   -0.068276848825327   │
│ 5.785714285714285  │      9      │  0.028945455462706497  │
│ 5.785714285714285  │     10      │  0.02116376699652612   │
│ 5.785714285714285  │     11      │  0.02116376699652612   │
│ 5.785714285714285  │     12      │  0.005958440633584836  │
│ 6.191836734693877  │      2      │  -0.06342855655026242  │
│ 6.191836734693877  │      3      │  0.042007571335556985  │
│ 6.191836734693877  │      4      │  0.03285494758484445   │
│ 6.191836734693877  │      5      │  0.036049440554928404  │
│ 6.191836734693877  │      6      │   0.071635542983881    │
│ 6.191836734693877  │      7      │  0.08474744226546951   │
│ 6.191836734693877  │      8      │  0.04287619029232182   │
│ 6.191836734693877  │      9      │ -0.019409861387179837  │
│ 6.191836734693877  │     10      │  -0.01096473073600401  │
│ 6.191836734693877  │     11      │  0.03913030913720082   │
│ 6.191836734693877  │     12      │  0.03571091524892712   │
│ 6.191836734693877  │     13      │  0.03571091524892712   │
│ 6.191836734693877  │     14      │  0.03571091524892712   │
│ 6.597959183673469  │      2      │ -0.027949730646737607  │
│ 6.597959183673469  │      3      │  0.04076525191369833   │
│ 6.597959183673469  │      4      │  0.08229753374928515   │
│ 6.597959183673469  │      5      │  0.07606822886436705   │
│ 6.597959183673469  │      6      │   0.0987181341233159   │
│ 6.597959183673469  │      7      │  0.06174802082615353   │
│ 6.597959183673469  │      8      │  0.04954006691986782   │
│ 6.597959183673469  │      9      │  0.09800755312708587   │
│ 6.597959183673469  │     10      │  0.09800755312708587   │
│ 6.597959183673469  │     11      │  0.09525113399850076   │
│ 6.597959183673469  │     12      │  0.09136302784761749   │
│ 6.597959183673469  │     13      │  0.00732967687956513   │
│ 6.597959183673469  │     14      │  0.006004419588525062  │
│ 7.004081632653061  │      2      │ -0.022029476681170806  │
│ 7.004081632653061  │      3      │  0.012734464369104713  │
│ 7.004081632653061  │      4      │  0.10084130871565712   │
│ 7.004081632653061  │      5      │  0.10704618412674562   │
│ 7.004081632653061  │      6      │  0.12008215234064079   │
│ 7.004081632653061  │      7      │  0.11336118364818677   │
│ 7.004081632653061  │      8   

# Função de Otimização do AGNES

Define a função que é usada para encontrar o número ideal de clusters e método de ligação para Agglomerative Clustering (AGNES) em um determinado conjunto de dados. AGNES é um tipo de método de agrupamento hierárquico que mescla o par mais próximo de clusters em cada etapa. Resultando em todos os possiveis valores de clusters e suas respectivas inércias.

In [162]:
def optimal_agnes(df, start=2, end=11, linkage_methods=['ward', 'complete', 'average', 'single']):
    '''
    Esta função plota o escore de silhueta para um intervalo de números de clusters e métodos de ligação.
    df: DataFrame. Seus dados.
    start: int. Número inicial de clusters que você deseja testar.
    end: int. Número final de clusters que você deseja testar.
    linkage_methods: list. Métodos de ligação que você deseja testar.
    '''
    
    # Escore de silhueta
    silhouette = {}
    
    for linkage in linkage_methods:
        for n_clusters in range(start, end):
            agnes = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage).fit(df)
            labels = agnes.labels_
            silhouette[(linkage, n_clusters)] = silhouette_score(df, labels, metric='euclidean')
    
    # Plota um gráfico com o plotly com todos os métodos de ligação na mesma figura
    fig = go.Figure()

    for linkage in linkage_methods:
        fig.add_trace(go.Scatter(x=list(range(start, end)), y=[silhouette[(linkage, i)] for i in range(start, end)], 
                         mode='lines+markers', name=linkage))
        
    fig.update_layout(title_text="Gráfico de análise - Agglomerative Clustering")

    fig.update_layout(
        autosize=False,
        width=1400,
        height=800,
    )

    fig.show()

    # Com o rich, cria uma tabela com os resultados
    table = Table(title="Resultados")
    table.add_column("Linkage", justify="center", style="cyan", no_wrap=True)
    table.add_column("Clusters", justify="center", style="magenta", no_wrap=True)
    table.add_column("Silhouette Score", justify="center", style="green", no_wrap=True)
    
    for key, value in silhouette.items():
        table.add_row(str(key[0]), str(key[1]), str(value))
        
    console = Console()
    console.print(table)

# Uso da função
optimal_agnes(df)

                  Resultados                  
┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Linkage  ┃ Clusters ┃   Silhouette Score   ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│   ward   │    2     │ 0.18807453869822213  │
│   ward   │    3     │  0.1541185392067335  │
│   ward   │    4     │ 0.13606889845188838  │
│   ward   │    5     │ 0.14265412776493583  │
│   ward   │    6     │  0.147140020930462   │
│   ward   │    7     │ 0.14131498396572326  │
│   ward   │    8     │  0.1367343647407059  │
│   ward   │    9     │ 0.13667043382091312  │
│   ward   │    10    │ 0.13329588898684172  │
│ complete │    2     │  0.1295440791425587  │
│ complete │    3     │  0.1189812829284411  │
│ complete │    4     │ 0.10992661001137863  │
│ complete │    5     │ 0.10918605684795538  │
│ complete │    6     │ 0.11502817827984325  │
│ complete │    7     │ 0.10347241148997002  │
│ complete │    8     │ 0.10987107153998671  │
│ complete │    9     │  0.1144138361668523  │
│ complete │    10    │ 0.12020135166709317  │
│ average  │    2     │  0.1729488217095168  │
│ average  │    3     │ 0.12547685764897568  │
│ average  │    4     │  0.1598694403754856  │
│ average  │    5     │  0.1682867491308201  │
│ average  │    6     │ 0.15627024933610414  │
│ average  │    7     │  0.1325176019057598  │
│ average  │    8     │ 0.13143638907043112  │
│ average  │    9     │ 0.11608483044547785  │
│ average  │    10    │ 0.11436281718371485  │
│  single  │    2     │ 0.09513887255052131  │
│  single  │    3     │ 0.04057676324073076  │
│  single  │    4     │ 0.035018886770669035 │
│  single  │    5     │ -0.03880366922664436 │
│  single  │    6     │ -0.04621814845182896 │
│  single  │    7     │ -0.07763316642062353 │
│  single  │    8     │ -0.10509226401365336 │
│  single  │    9     │ -0.15807225725602397 │
│  single  │    10    │ -0.15823384190745754 │
└──────────┴──────────┴──────────────────────┘

---

# Técnicas utilizadas para redução de dimensionalidade

## PCA (Principal Component Analysis)

PCA, ou Análise de Componentes Principais, é uma técnica de redução de dimensionalidade usada em aprendizado de máquina e estatística. O objetivo do PCA é transformar um conjunto de variáveis possivelmente correlacionadas em um conjunto menor de variáveis não correlacionadas chamadas componentes principais.

### No código

```python

pca = PCA(n_components=2)

```
- n_components: número de componentes principais a serem mantidos, aqui, um objeto PCA é criado, especificando que queremos reduzir nosso conjunto de dados para 2 componentes principais

```python
principalComponents = pca.fit_transform(df)
```

- O método fit_transform() é chamado no objeto PCA, que calcula os componentes principais do DataFrame df e usa esses componentes para transformar df em um novo espaço de dados. O resultado é um array numpy principalComponents que contém os dados transformados



---

# Método do KMeans

O KMeans é um método de clusterização que tem como objetivo dividir um conjunto de dados em K grupos, onde K é um número pré-definido

In [163]:
# Define a função para o KMeans
def KMeans_Trainer(df):
    
    # Determina o número de clusters do K-Means
    n_clusters = 5

    # Determina o máximo de iteração
    max_iter = 100
    
    # Define o modelo
    model = KMeans(n_clusters=n_clusters, max_iter=max_iter, random_state=42)    
    
    # Define os dados de treino
    model.fit(df)
    
    # Define a opinião do modelo
    X = model.labels_

    # Printa a opinião do modelo com o rich
    console = Console()
    table = Table(title="Opinião do Modelo K-Means")
    table.add_column("Cluster", style="cyan", justify="center")
    table.add_column("Opinião", style="magenta", justify="center")
    for i in range(n_clusters):
        table.add_row(str(i), str(np.sum(X == i)))
    console.print(table)
    
    # Printa o X com o rich
    console = Console()
    console.print(X)
    
    # Plota o gráfico de dispersão dos dados K-Means
    
    # Supondo que meu dataframe seja o 'X' e que 'y_kmeans' seja o resultado do agrupamento
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)

    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

    # Adiciona a coluna de agrupamento
    principalDf['cluster'] = X

    # Ordena o DataFrame pelo valor do cluster (Arruma a legenda do gráfico para que os clusters fiquem em ordem crescente)
    principalDf = principalDf.sort_values(by='cluster')

    # Transforma na df todos o cluster, os que era 0 viram 1, 1 vira 2 e assim por diante
    principalDf['cluster'] = principalDf['cluster'] + 1

    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', symbol='cluster', size='cluster', color='cluster')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)

    # Atualiza o X e Y do gráfico retirando-os
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    
    # Remove a barra de cores
    fig.update_layout(coloraxis_showscale=False)
    
    # Retira o fundo do gráfico (grid) colocando-o branco
    fig.update_layout(plot_bgcolor='white')
    
    # Retira o axis do gráfico
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False)
    fig.update_yaxes(showline=False, showgrid=False, zeroline=False)
    
    # Retira os numeros do eixo X e Y
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    # Organiza a legenda em ordem crescente de clusters
    fig.for_each_trace(lambda t: t.update(name='Cluster ' + str(t.name)))

    # Configura a legenda
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=20,
                color='black'
            ),
            bordercolor='Black',
            borderwidth=2
        ),
        
        # Padding para tirar o espaço em branco em volta do gráfico
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    # Retira o titulo da legenda
    fig.update_layout(legend_title_text='')

    # Exibe o gráfico
    fig.show()
    
    # Soma dos quadrados das distâncias
    sse = model.inertia_
    
    # Calcula a coesao
    cohesion = mt.sqrt(model.inertia_)/model.n_clusters
    
    # Calcula o coeficiente de silhueta
    silhouette_kmeans = silhouette_score(df, X)
    
    # Rand score do KMeans
    rand_kmeans = metrics.adjusted_rand_score(df['clusters'], X)
    
    # Homogeneidade
    homogeneity = metrics.homogeneity_score(df['clusters'], X)
    
    # Completude
    completeness = metrics.completeness_score(df['clusters'], X)
    
    # Matriz de confusão
    confusion_matrix = metrics.cluster.contingency_matrix(df['clusters'], X)
    
    
    # Printa 
    console = Console()
    table = Table(title="Resultados do K-Means")
    table.add_column("SSE", style="cyan", justify="center")
    table.add_column("Coesão", style="magenta", justify="center")
    table.add_column("Silhouette Score", style="green", justify="center")
    table.add_column("Homogeneidade", style="blue", justify="center")
    table.add_column("Rand Score", style="yellow", justify="center")
    table.add_column("Completude", style="red", justify="center")
    
    
    table.add_row(str(sse), str(cohesion), str(silhouette_kmeans), str(homogeneity), str(rand_kmeans), str(completeness))
    
    console.print(table)

    # Printa a matriz de confusão
    print("Matriz de Confusão")
    print(confusion_matrix)
    
    
    return silhouette_kmeans
    
# Chamada da função KMeans
silhouette_kmeans = KMeans_Trainer(df)


  Opinião do Modelo  
       K-Means       
┏━━━━━━━━━┳━━━━━━━━━┓
┃ Cluster ┃ Opinião ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│    0    │   18    │
│    1    │   54    │
│    2    │   46    │
│    3    │   48    │
│    4    │   17    │
└─────────┴─────────┘

[3 1 2 1 2 3 1 3 2 3 2 1 2 2 2 2 3 2 2 2 2 1 2 3 1 1 1 3 3 2 1 3 4 2 1 3 1
 1 1 2 0 2 3 1 3 3 3 3 1 4 2 0 1 3 2 2 2 1 1 3 1 3 2 1 1 2 2 1 0 1 4 1 4 3
 0 0 3 4 3 1 1 3 1 4 1 3 1 1 4 3 3 3 1 3 2 1 3 1 1 2 1 3 3 1 1 0 1 3 3 3 1
 3 3 4 4 3 0 1 2 1 1 3 1 3 1 2 3 4 0 4 4 1 3 2 2 4 2 1 2 2 1 3 1 0 1 2 0 1
 2 4 2 3 0 1 2 0 2 4 1 0 2 0 2 1 2 3 3 4 3 0 3 0 2 3 1 2 2 0 4 2 2 0 3]

                                               Resultados do K-Means                                               
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃       SSE        ┃      Coesão      ┃ Silhouette Score ┃  Homogeneidade   ┃    Rand Score    ┃    Completude    ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 7905.8155163309… │ 17.782930597998… │ 0.1490572577991… │ 0.5738485167753… │ 0.4787819914895… │ 0.8544080637867… │
└──────────────────┴──────────────────┴──────────────────┴──────────────────┴──────────────────┴──────────────────┘

Matriz de Confusão
[[10  0  0  0  0]
 [ 0 30  0  0  0]
 [ 7  0  0  0  5]
 [ 0  1  0  0 11]
 [ 0 23  0  2  0]
 [ 1  0  1 19  1]
 [ 0  0 13  0  0]
 [ 0  0  0 23  0]
 [ 0  0 18  0  0]
 [ 0  0 14  4  0]]


# Método do DBSCAN

O DBSCAN é um método de clusterização que tem como objetivo dividir um conjunto de dados em grupos de densidade, onde os grupos são formados por pontos que estão próximos uns dos outros

In [164]:
def DBSCAN_Trainer(df):
    
    # Define o EPS
    eps = 7
    
    # Define o número mínimo de amostras
    min_samples = 5
    
    # Define o modelo
    model = DBSCAN(eps=eps, min_samples=min_samples)
    
    # Define os dados de treino
    model.fit(df)
    
    # Define a opinião do modelo
    X = model.labels_

    # DBSCAN predict
    dbscan_labels = model.fit_predict(df)

    # Printa a opinião do modelo com o rich
    console = Console()
    table = Table(title="Opinião do Modelo DBSCAN")
    table.add_column("Cluster", style="cyan", justify="center")
    table.add_column("Opinião", style="magenta", justify="center")
    for i in range(len(np.unique(X))):
        table.add_row(str(i), str(np.sum(X == i)))
    console.print(table)
    
    # Printa o X com o rich
    console = Console()
    console.print(X)
    
    # Plota o gráfico de dispersão dos dados DBSCAN
    
    # Supondo que meu dataframe seja o 'X' e que 'y_kmeans' seja o resultado do agrupamento
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)

    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

    # Adiciona a coluna de agrupamento
    principalDf['cluster'] = X

    # Ordena o DataFrame pelo valor do cluster (Arruma a legenda do gráfico para que os clusters fiquem em ordem crescente)
    principalDf = principalDf.sort_values(by='cluster')

    # Transforma na df todos o cluster, os que era 0 viram 1, 1 vira 2 e assim por diante
    principalDf['cluster'] = principalDf['cluster'] + 1

    
    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', symbol='cluster', size='cluster', color='cluster')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)
    
    # Atualiza o X e Y do gráfico retirando-os
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    
    # Remove a barra de cores
    fig.update_layout(coloraxis_showscale=False)
    
    # Retira o fundo do gráfico (grid) colocando-o branco
    fig.update_layout(plot_bgcolor='white')

    # Retira o axis do gráfico
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False)
    fig.update_yaxes(showline=False, showgrid=False, zeroline=False)
    
    # Retira os numeros do eixo X e Y
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    # Organiza a legenda em ordem crescente de clusters
    fig.for_each_trace(lambda t: t.update(name='Cluster ' + str(t.name)))
    
    # Configura a legenda
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=20,
                color='black'
            ),
            bordercolor='Black',
            borderwidth=2
        ),
        
        # Padding para tirar o espaço em branco em volta do gráfico
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    # Retira o titulo da legenda
    fig.update_layout(legend_title_text='')
    
    # Exibe o gráfico
    fig.show()
    
    # Calcula o coeficiente de silhueta
    n_clusters = len(set(X))
    
    if n_clusters > 1:
        silhouette_dbscan = silhouette_score(df, X)
    else:
        silhouette_dbscan = 0
    

    # Calcula a coesão para o DBSCAN
    dbscan_cohesion = 0
    for cluster in np.unique(dbscan_labels):
        if cluster != -1:  # Ignora os pontos de ruído
            cluster_points = df[dbscan_labels == cluster]
            
            # Calcula o centróide do cluster
            centroid = np.mean(cluster_points, axis=0)
            
            # Calcula a distância de cada ponto ao centróide
            distances = euclidean_distances(cluster_points, [centroid])
            
            # Soma as distâncias para obter a coesão do cluster
            dbscan_cohesion += np.sum(distances)

    # Divide pela quantidade de clusters
    dbscan_cohesion /= len(np.unique(dbscan_labels))

    
    # Rand score do DBSCAN
    rand_dbscan = metrics.adjusted_rand_score(df['clusters'], X)
    
    # Homogeneidade
    homogeneity = metrics.homogeneity_score(df['clusters'], X)
    
    # Completude
    completeness = metrics.completeness_score(df['clusters'], X)
    
    # Matriz de confusão
    confusion_matrix = metrics.cluster.contingency_matrix(df['clusters'], X)
    
    # Printa
    console = Console()
    table = Table(title="Resultados do DBSCAN")
    table.add_column("Coesão", style="cyan", justify="center")
    table.add_column("Silhouette Score", style="magenta", justify="center")
    table.add_column("Rand Score", style="green", justify="center")
    table.add_column("Homogeneidade", style="blue", justify="center")
    table.add_column("Completude", style="yellow", justify="center")
     
    table.add_row(str(dbscan_cohesion), str(silhouette_dbscan), str(rand_dbscan), str(homogeneity), str(completeness))
    
    console.print(table)
    
    # Printa a matriz de confusão
    print("Matriz de Confusão")
    print(confusion_matrix)
    
    return silhouette_dbscan

# Chamada da função DBSCAN
silhouette_dbscan = DBSCAN_Trainer(df)

  Opinião do Modelo  
       DBSCAN        
┏━━━━━━━━━┳━━━━━━━━━┓
┃ Cluster ┃ Opinião ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│    0    │   138   │
│    1    │   18    │
│    2    │    5    │
│    3    │    0    │
└─────────┴─────────┘

[ 0  0  0  0  0  0  0  0 -1  0  2  0  2  1  1  1  0 -1  1  1 -1  0  1  0
  0  0  0  0  0  1  0  0  0 -1  0  0  0  0  0  1  0  2  0  0  0  0  0  0
  0  0 -1  0  0  0 -1 -1 -1  0  0  0 -1  0 -1  0  0 -1  0  0  0  0  0 -1
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  1  0 -1  0 -1 -1  0  0  0  0
  0  1  0  0 -1  0  0  0 -1  0  1  0  1  0  0  0  1 -1 -1 -1  2  0  0  0
  0  0  0  0  2  0  0  1  0  0  0 -1  1  0  0]

                                            Resultados do DBSCAN                                             
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃       Coesão       ┃  Silhouette Score   ┃     Rand Score      ┃    Homogeneidade    ┃     Completude     ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ 296.92225981387895 │ 0.10704618412674562 │ 0.08306641764548961 │ 0.20245756223544178 │ 0.5721937874596295 │
└────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┴────────────────────┘

Matriz de Confusão
[[ 2  8  0  0]
 [ 0 30  0  0]
 [ 0 12  0  0]
 [ 0 12  0  0]
 [ 3 22  0  0]
 [ 1 21  0  0]
 [ 5  2  6  0]
 [ 0 23  0  0]
 [ 6  0 12  0]
 [ 5  8  0  5]]


# Método do AGNES

O AGNES é um método de clusterização que tem como objetivo dividir um conjunto de dados em grupos de hierarquia, onde os grupos são formados por pontos que estão próximos uns dos outros

In [165]:
def AGNES_Trainer(df):
    
    # Define o número de clusters
    n_clusters = 5
    
    # Define o método de ligação
    linkage = 'average'
    
    # Define o modelo
    model = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    
    # Define os dados de treino
    model.fit(df)
    
    # Define a opinião do modelo
    X = model.labels_
    
    # AGNES predict
    agnes_labels = model.fit_predict(df)

    # Printa a opinião do modelo com o rich
    console = Console()
    table = Table(title="Opinião do Modelo AGNES")
    table.add_column("Cluster", style="cyan", justify="center")
    table.add_column("Opinião", style="magenta", justify="center")
    for i in range(n_clusters):
        table.add_row(str(i), str(np.sum(X == i)))
    console.print(table)
    
    # Printa o X com o rich
    console = Console()
    console.print(X)
    
    # Plota o gráfico de dispersão dos dados AGNES
    
    # Supondo que meu dataframe seja o 'X' e que 'y_kmeans' seja o resultado do agrupamento
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)

    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

    # Adiciona a coluna de agrupamento
    principalDf['cluster'] = X

    # Ordena o DataFrame pelo valor do cluster (Arruma a legenda do gráfico para que os clusters fiquem em ordem crescente)
    principalDf = principalDf.sort_values(by='cluster')
    
    # Transforma na df todos o cluster, os que era 0 viram 1, 1 vira 2 e assim por diante
    principalDf['cluster'] = principalDf['cluster'] + 1

    # Plota o gráfico de dispersão usando o matplotlib
    fig = px.scatter(principalDf, x='principal component 1', y='principal component 2', symbol='cluster', size='cluster', color='cluster')
    
    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)
    
    # Atualiza o X e Y do gráfico retirando-os
    fig.update_xaxes(title_text='')
    fig.update_yaxes(title_text='')
    
    # Remove a barra de cores
    fig.update_layout(coloraxis_showscale=False)
    
    # Retira o fundo do gráfico (grid) colocando-o branco
    fig.update_layout(plot_bgcolor='white')

    # Retira o axis do gráfico
    fig.update_xaxes(showline=False, showgrid=False, zeroline=False)   
    fig.update_yaxes(showline=False, showgrid=False, zeroline=False)
    
    # Retira os numeros do eixo X e Y
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    
    # Organiza a legenda em ordem crescente de clusters
    fig.for_each_trace(lambda t: t.update(name='Cluster ' + str(t.name)))
    
    # Configura a legenda
    fig.update_layout(
        legend=dict(
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=20,
                color='black'
            ),
            bordercolor='Black',
            borderwidth=2
        ),
        
        # Padding para tirar o espaço em branco em volta do gráfico
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    # Retira o titulo da legenda
    fig.update_layout(legend_title_text='')
    
    # Exibe o gráfico
    fig.show()
    
    # Calcula a coesão para o AGNES
    agnes_cohesion = 0
    for cluster in np.unique(agnes_labels):
        cluster_points = df[agnes_labels == cluster]
        # Calcula o centróide do cluster
        centroid = np.mean(cluster_points, axis=0)
        # Calcula a distância de cada ponto ao centróide
        distances = euclidean_distances(cluster_points, [centroid])
        # Soma as distâncias para obter a coesão do cluster
        agnes_cohesion += np.sum(distances)

    # Divide pela quantidade de clusters
    agnes_cohesion /= len(np.unique(agnes_labels))  # Conta os clusters
    
    # Calcula o coeficiente de silhueta
    silhouette_agnes = silhouette_score(df, X)
    
    # Rand score do AGNES
    rand_agnes = metrics.adjusted_rand_score(df['clusters'], X)
    
    # Homogeneidade
    homogeneity = metrics.homogeneity_score(df['clusters'], X)
    
    # Completude
    completeness = metrics.completeness_score(df['clusters'], X)
    
    # Matriz de confusão
    confusion_matrix = metrics.cluster.contingency_matrix(df['clusters'], X)
    
    # Printa
    console = Console()
    table = Table(title="Resultados do AGNES")
    table.add_column("Coesão", style="cyan", justify="center")
    table.add_column("Silhouette Score", style="magenta", justify="center")
    table.add_column("Rand Score", style="green", justify="center")
    table.add_column("Homogeneidade", style="blue", justify="center")
    table.add_column("Completude", style="yellow", justify="center")
    
    table.add_row(str(agnes_cohesion), str(silhouette_agnes), str(rand_agnes), str(homogeneity), str(completeness))
    
    console.print(table)
    
    # Printa a matriz de confusão
    print("Matriz de Confusão")
    print(confusion_matrix)
    
    return silhouette_agnes
    
# Chamada da função AGNES
silhouette_agnes = AGNES_Trainer(df)

  Opinião do Modelo  
        AGNES        
┏━━━━━━━━━┳━━━━━━━━━┓
┃ Cluster ┃ Opinião ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│    0    │   51    │
│    1    │   59    │
│    2    │   33    │
│    3    │   24    │
│    4    │   16    │
└─────────┴─────────┘

[4 2 4 1 4 4 1 0 0 0 4 1 4 3 3 3 2 0 3 3 3 1 0 0 1 1 1 4 0 3 1 0 2 0 1 0 1
 1 1 3 1 4 0 1 4 0 0 1 1 2 4 2 1 0 0 4 0 1 1 0 1 0 0 1 1 3 4 1 2 1 2 1 2 0
 2 2 0 2 0 1 1 2 1 1 1 0 1 1 0 0 0 0 1 0 3 1 0 1 1 3 1 0 4 1 1 2 1 0 4 0 1
 0 0 2 2 0 2 1 3 1 1 0 1 0 1 0 0 2 2 2 1 1 0 3 3 2 0 1 3 3 1 1 1 2 1 3 2 1
 3 2 0 0 2 1 3 2 3 2 1 2 3 2 3 1 4 0 0 2 0 2 0 2 4 0 1 3 0 2 2 1 3 0 0]

                                           Resultados do AGNES                                            
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃       Coesão       ┃  Silhouette Score  ┃     Rand Score     ┃   Homogeneidade    ┃     Completude     ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ 236.83416034084476 │ 0.1682867491308201 │ 0.4075158868013831 │ 0.4932279885120558 │ 0.7332698872774241 │
└────────────────────┴────────────────────┴────────────────────┴────────────────────┴────────────────────┘

Matriz de Confusão
[[ 0  0 10  0  0]
 [ 0 29  1  0  0]
 [ 0  1 11  0  0]
 [ 0  3  9  0  0]
 [ 1 24  0  0  0]
 [18  1  2  1  0]
 [ 5  1  0  7  0]
 [21  0  0  0  2]
 [ 4  0  0 14  0]
 [ 2  0  0  2 14]]


---

# Técnicas extras

## O que são?

Tanto o t-SNE (t-Distributed Stochastic Neighbor Embedding) quanto o UMAP (Uniform Manifold Approximation and Projection) são técnicas de redução de dimensionalidade que são comumente usadas para visualização de dados de alta dimensão. Ambos seguem um processo semelhante onde começam calculando probabilidades de alta dimensão p, depois probabilidades de baixa dimensão q, seguido pelo cálculo da função de custo C(p,q) comparando as diferenças entre as probabilidade

No entanto, existem algumas diferenças importantes entre os dois:

- Velocidade e Escalabilidade: O UMAP geralmente é mais rápido que o t-SNE e pode lidar melhor com conjuntos de dados maiores.
- Preservação da Estrutura Global: Enquanto o t-SNE se concentra em preservar a estrutura local dos dados (ou seja, mantém pontos que são próximos no espaço de alta dimensão próximos no espaço de baixa dimensão), o UMAP tenta preservar tanto a estrutura local quanto a global dos dados. Isso significa que o UMAP pode preservar melhor as relações de distância entre os clusters.
- Reprodutibilidade: O t-SNE é uma técnica estocástica, o que significa que você pode obter resultados ligeiramente diferentes cada vez que você executar o código, mesmo com a mesma semente aleatória. Por outro lado, o UMAP é mais consistente e produz resultados mais reprodutíveis.
- Uso de Hiperparâmetros: Ambos os métodos têm hiperparâmetros que podem ser ajustados para alterar o resultado da redução de dimensionalidade. No entanto, a interpretação e o ajuste desses hiperparâmetros podem ser menos intuitivos no t-SNE do que no UMAP.

In [166]:
from sklearn.manifold import TSNE

def TSNE_trainer(df):
    
    # Define o modelo
    model = TSNE(n_components=2, random_state=0)
    
    # Define os dados de treino
    X = model.fit_transform(df)
    
    # Plota o gráfico de dispersão dos dados TSNE
    
    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = X, columns = ['TSNE 1', 'TSNE 2'])

    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='TSNE 1', y='TSNE 2')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)

    # Atualiza o X e Y do gráfico
    fig.update_xaxes(title_text='TSNE 1')
    fig.update_yaxes(title_text='TSNE 2')
    
    # Exibe o gráfico
    fig.show()
    
# Chamada da função TSNE
TSNE_trainer(df)

In [167]:
from umap import UMAP

def UMAP_trainer(df):
    
    # Define o modelo
    model = UMAP(n_components=2, random_state=0)
    
    # Define os dados de treino
    X = model.fit_transform(df)
    
    # Plota o gráfico de dispersão dos dados UMAP
    
    # Cria um dataframe com os componentes principais
    principalDf = pd.DataFrame(data = X, columns = ['UMAP 1', 'UMAP 2'])

    # Plota o gráfico de dispersão
    fig = px.scatter(principalDf, x='UMAP 1', y='UMAP 2')

    # Atualiza o layout do gráfico
    fig.update_layout(
        width=800, 
        height=800)

    # Atualiza o X e Y do gráfico
    fig.update_xaxes(title_text='UMAP 1')
    fig.update_yaxes(title_text='UMAP 2')
    
    # Exibe o gráfico
    fig.show()
    
# Chamada da função UMAP
UMAP_trainer(df)

ModuleNotFoundError: No module named 'umap'